In [1]:
# загружаем словарь
# !!! словарь должен быть в той же папке, что и этот файл !!!
from lxml import etree
with open('fra-deu.tei', encoding='utf-8') as f:
    text = f.read()
root = etree.fromstring(text)

In [2]:
# найдём все узлы с тэгом entry
entries = root.xpath('.//entry')

# проверяем количество вхождений
print(len(entries), 'entries')

29337 entries


In [3]:
# загружаем датафрейм с формами глаголов третьей группы
import pandas as pd
df = pd.read_csv('third_group_verbs_table.csv')
third_group_verbs_list = [i for i in df.lemma]

In [4]:
# функция, распределяющая все глаголы по группам
first_group = []
second_group = []
lost_verbs = []
unidentified_verbs = []
def identify(w):
    if w in third_group_verbs_list:
        return ['3nd group',
                df.présent[df.lemma==w][third_group_verbs_list.index(w)],
                df.passécomposé[df.lemma==w][third_group_verbs_list.index(w)],
                df.futur[df.lemma==w][third_group_verbs_list.index(w)],
                df.subjonctif[df.lemma==w][third_group_verbs_list.index(w)]]
    elif (w[:6] == 'se la '):
        return ['is conjugated like ' + w[6:].split(' ')[0]]
    elif (w[:3] == 'se '):
        return ['is conjugated like ' + w[3:].split(' ')[0]]
    elif (w[:5] == "s’en "):
        return ['is conjugated like ' + w[5:].split(' ')[0]]
    elif (w[:2] == "s’"):
        return ['is conjugated like ' + w[2:].split(' ')[0]]
    elif (w[-2:] == 'ir' or w[-2:] == 'ïr')  and ' ' not in w:
        second_group.append(w)
        return ['2nd group']
    elif w[-2:] == 'er' and ' ' not in w:
        first_group.append(w)
        return ['1st group']
    elif ' ' not in w:
        return []
        lost_verbs.append(w)
    else:
        return []
        unidentified_verbs.append(w)

In [5]:
print(identify(str(entries[25158].xpath('.//orth/text()'))[2:-2]))

['3nd group', 'je sors, nous sortons', 'je suis sorti', 'je sortirai', 'que je sorte']


In [14]:
#all_verbs_elements_list = [i for i in entries if i.xpath('.//pos/text()') == ['v']]
#for v in all_verbs_elements_list:
#    w = str(v.xpath('.//orth/text()'))[2:-2]
#    print(w, identify(w))

In [6]:
# прогоняем identify() по всем глаголам и добавляем нужные поля
for i in entries:
    if i.xpath('.//pos/text()') == ['v']:
        lst = identify(str(i.xpath('.//orth/text()'))[2:-2])
        if len(lst)>0:
            etree.SubElement(i.xpath('.//gramGrp')[0], 'subc').text = lst[0]
            if len(lst)>1:
                new = etree.SubElement(i, 'form')
                new.set('type', 'inflected')
                for j in lst[1:]:
                    etree.SubElement(new, 'orth').text = j
                i.append(new)

In [7]:
# наводим красоту
def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

indent(root)

In [8]:
# проверяем
print(etree.tostring(entries[-103], encoding='utf-8', pretty_print=True).decode())
print(etree.tostring(entries[26689], encoding='utf-8', pretty_print=True).decode())

<entry>
        <form>
          <orth>être</orth>
        </form>
        <gramGrp>
          <pos>v</pos>
          <subc>3nd group</subc>
        </gramGrp>
        <sense n="1">
          <usg type="hint">Copule</usg>
          <cit type="trans" xml:lang="de">
            <quote>sein</quote>
          </cit>
        </sense>
        <sense n="2">
          <usg type="hint">Se trouver dans une location, une situation</usg>
          <cit type="trans" xml:lang="de">
            <quote>sein</quote>
          </cit>
        </sense>
        <sense n="3">
          <usg type="hint">Auxiliaire passif</usg>
          <cit type="trans" xml:lang="de">
            <quote>sein</quote>
          </cit>
        </sense>
        <form type="inflected">
          <orth>je suis, nous sommes</orth>
          <orth>j’ai été</orth>
          <orth>je serai</orth>
          <orth>que je sois, que nous soyons</orth>
        </form>
      </entry>
      

<entry>
        <form>
          <orth>tourner</

In [9]:
# записываем результат в файл
et = etree.ElementTree(root)
et.write('fra-deu_parsed.tei', encoding = 'utf-8', pretty_print=True)